<a href="https://colab.research.google.com/github/pkmariya/Scaler01/blob/master/LoanTap_CaseStudy_Mariya_DSMLJune22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

Given a set of attributes for an Individual, determine if a credit line should be extended to them. If so, what should the repayment terms be in business recommendations?

# Import required Libraries

In [67]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import datetime

In [9]:
!pip install simple_colors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from simple_colors import *

# Load the Dataset

In [2]:
# Load the dataset
data_df = pd.read_csv('https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/003/549/original/logistic_regression.csv?1651045921')

# Statistical Summary of the Dataset

In [ ]:
# Meta data of dataset
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394275 non-null  object 
 15  

In [ ]:
# Description of the dataset
data_df.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,396030.000000,396030.000000,396030.000000,3.960300e+05,396030.000000,396030.000000,396030.000000,3.960300e+05,395754.000000,396030.000000,358235.000000,395495.000000
mean,14113.888089,13.639400,431.849698,7.420318e+04,17.379514,11.311153,0.178191,1.584454e+04,53.791749,25.414744,1.813991,0.121648
std,8357.441341,4.472157,250.727790,6.163762e+04,18.019092,5.137649,0.530671,2.059184e+04,24.452193,11.886991,2.147930,0.356174
min,500.000000,5.320000,16.080000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000
25%,8000.000000,10.490000,250.330000,4.500000e+04,11.280000,8.000000,0.000000,6.025000e+03,35.800000,17.000000,0.000000,0.000000
50%,12000.000000,13.330000,375.430000,6.400000e+04,16.910000,10.000000,0.000000,1.118100e+04,54.800000,24.000000,1.000000,0.000000
75%,20000.000000,16.490000,567.300000,9.000000e+04,22.980000,14.000000,0.000000,1.962000e+04,72.900000,32.000000,3.000000,0.000000
max,40000.000000,30.990000,1533.810000,8.706582e+06,9999.000000,90.000000,86.000000,1.743266e+06,892.300000,151.000000,34.000000,8.000000


In [ ]:
data_df.dtypes

loan_amnt               float64
term                     object
int_rate                float64
installment             float64
grade                    object
sub_grade                object
emp_title                object
emp_length               object
home_ownership           object
annual_inc              float64
verification_status      object
issue_d                  object
loan_status              object
purpose                  object
title                    object
dti                     float64
earliest_cr_line         object
open_acc                float64
pub_rec                 float64
revol_bal               float64
revol_util              float64
total_acc               float64
initial_list_status      object
application_type         object
mort_acc                float64
pub_rec_bankruptcies    float64
address                  object
dtype: object

In [ ]:
# Shape of the dataset
print("Given dataset contains {0} Rows and {1} Columns ".format(data_df.shape[0], data_df.shape[1]))

Given dataset contains 396030 Rows and 27 Columns 


In [ ]:
# Unique value counts
data_df.nunique()

loan_amnt                 1397
term                         2
int_rate                   566
installment              55706
grade                        7
sub_grade                   35
emp_title               173105
emp_length                  11
home_ownership               6
annual_inc               27197
verification_status          3
issue_d                    115
loan_status                  2
purpose                     14
title                    48817
dti                       4262
earliest_cr_line           684
open_acc                    61
pub_rec                     20
revol_bal                55622
revol_util                1226
total_acc                  118
initial_list_status          2
application_type             3
mort_acc                    33
pub_rec_bankruptcies         9
address                 393700
dtype: int64

In [ ]:
# Value types and their unique counts of Categorical Variables
for col in data_df.astype('object').columns:
  if len(data_df[col].value_counts())<10:
    print(blue(col, ['bold']))
    print(data_df[col].value_counts().to_string(),'\n')

term
 36 months    302005
 60 months     94025 

grade
B    116018
C    105987
A     64187
D     63524
E     31488
F     11772
G      3054 

home_ownership
MORTGAGE    198348
RENT        159790
OWN          37746
OTHER          112
NONE            31
ANY              3 

verification_status
Verified           139563
Source Verified    131385
Not Verified       125082 

loan_status
Fully Paid     318357
Charged Off     77673 

initial_list_status
f    238066
w    157964 

application_type
INDIVIDUAL    395319
JOINT            425
DIRECT_PAY       286 

pub_rec_bankruptcies
0.0    350380
1.0     42790
2.0      1847
3.0       351
4.0        82
5.0        32
6.0         7
7.0         4
8.0         2 



#### **Observations**: 
*   Loan Term 36 months is more predominant than 60 months
*   Grade B and C has more counts than other grades
*   In terms of Ownership, 'Mortgage' tops the list
*   3/4th of the Customers have fully paid back the loan
*   "Individual" seems to be the popular amongst the application type  




In [ ]:
data_df.describe(include='object').T

,count,unique,top,freq
term,396030,2,36 months,302005
grade,396030,7,B,116018
sub_grade,396030,35,B3,26655
emp_title,373103,173105,Teacher,4389
emp_length,377729,11,10+ years,126041
home_ownership,396030,6,MORTGAGE,198348
verification_status,396030,3,Verified,139563
issue_d,396030,115,Oct-2014,14846
loan_status,396030,2,Fully Paid,318357
purpose,396030,14,debt_consolidation,234507


In [ ]:
# Top 5 Rows
data_df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"


# Data Pre-Processing and Data Engineering

### Missing Values check

In [46]:
# Method to check for Missing Values
def check_for_missing_values():
  output = data_df.columns[data_df.isna().any()]
  if output.size>0:
    print(blue("Following Columns have Missing Values \n",['bold']))
    print(blue("Column_Name \t Missing_Values_Count \t Percentage"))
    for col in data_df.columns:
      if data_df[col].isna().sum()!=0:
        print(col, " \t\t ", data_df[col].isna().sum(), " \t\t ", round(data_df[col].isnull().sum()/len(data_df.index)*100, 2))
  else:
    print("Given dataset doesn't have any missing values")
    print(data_df.isna().sum())

In [19]:
check_for_missing_values()

Following Columns have Missing Values 

Column_Name 	 Missing_Values_Count 	 Percentage
emp_title  	  22927  		  5.79
emp_length  	  18301  		  4.62
title  	  1755  		  0.44
revol_util  	  276  		  0.07
mort_acc  	  37795  		  9.54
pub_rec_bankruptcies  	  535  		  0.14


#### **Observation**: 
*   Clearly 'emp_title', 'emp_length' and 'mort_acc' have significant null values. Remaining features have negligible null values which can be dropped.
*   Null values in 'emp_title' can be filled with 'Unknown title'
*   Null values in 'emp_length' can be imputed with mean value
*   'mort_acc' is number of mortgage accounts. We have to impute it with median to it's null values

In [27]:
# Imputing missing values of 'emp_title' with 'unknown_title'
data_df['emp_title'].fillna('unknown_title', inplace=True)

In [39]:
# Imputing null values of 'emp_length' with forward fill method
data_df['emp_length'] = data_df['emp_length'].fillna(method='ffill')

In [25]:
# Filling null values of 'mort_acc' with median value of 'mort_acc'
data_df['mort_acc'].fillna(data_df['mort_acc'].median(), inplace=True)

In [47]:
check_for_missing_values()

Following Columns have Missing Values 

Column_Name 	 Missing_Values_Count 	 Percentage
title  		  1755  		  0.44
revol_util  		  276  		  0.07
pub_rec_bankruptcies  		  535  		  0.14


In [48]:
# As the above columns have compartively less null values, these can be dropped. 
data_df = data_df.dropna()

In [49]:
check_for_missing_values()

Given dataset doesn't have any missing values
loan_amnt               0
term                    0
int_rate                0
installment             0
grade                   0
sub_grade               0
emp_title               0
emp_length              0
home_ownership          0
annual_inc              0
verification_status     0
issue_d                 0
loan_status             0
purpose                 0
title                   0
dti                     0
earliest_cr_line        0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
initial_list_status     0
application_type        0
mort_acc                0
pub_rec_bankruptcies    0
address                 0
dtype: int64


In [ ]:
# Check for duplicates
dup_rows = data_df[data_df.duplicated()]
print(blue("# of duplicate rows: {0} ".format(dup_rows.shape[0])))

# of duplicate rows: 0 


In [50]:
data_df['emp_length'].value_counts()

10+ years    131320
2 years       37294
< 1 year      33030
3 years       32987
5 years       27662
1 year        26856
4 years       24908
7 years       21766
6 years       21698
8 years       19979
9 years       15965
Name: emp_length, dtype: int64

In [51]:
# Convert the emp_length to a numerical column
# Before converting the datatype, let's make <1 yrs as 0 years and 10+ yrs to 10 years
data_df['emp_length'] = data_df['emp_length'].replace(['< 1 year'], '0 year')
data_df['emp_length'] = data_df['emp_length'].replace(['10+ years'], '10 years')

In [54]:
data_df['emp_length'].value_counts()

10 years    131320
2 years      37294
0 year       33030
3 years      32987
5 years      27662
1 year       26856
4 years      24908
7 years      21766
6 years      21698
8 years      19979
9 years      15965
Name: emp_length, dtype: int64

In [55]:
data_df[['emp_duration', 'emp_years']] = data_df['emp_length'].str.split(' ', expand=True)

In [57]:
data_df.drop(['emp_length', 'emp_years'], axis='columns', inplace=True)

In [59]:
data_df['emp_duration'].value_counts()

10    131320
2      37294
0      33030
3      32987
5      27662
1      26856
4      24908
7      21766
6      21698
8      19979
9      15965
Name: emp_duration, dtype: int64

In [60]:
data_df['emp_duration'] = data_df['emp_duration'].astype(int)

In [62]:
data_df.describe(include='object').T

,count,unique,top,freq
term,393465,2,36 months,300024
grade,393465,7,B,115395
sub_grade,393465,35,B3,26518
emp_title,393465,172227,unknown_title,22668
home_ownership,393465,6,MORTGAGE,197110
verification_status,393465,3,Verified,138867
issue_d,393465,112,Oct-2014,14838
loan_status,393465,2,Fully Paid,316271
purpose,393465,14,debt_consolidation,233108
title,393465,48472,Debt consolidation,152392


In [64]:
# Let's convert 'issue_dt' and 'earliest_cr_line' into datatime type
data_df['issue_d'] = data_df['issue_d'].astype('datetime64[ns]')
data_df['earliest_cr_line'] = data_df['earliest_cr_line'].astype('datetime64[ns]')

In [68]:
now = datetime.datetime.now()
data_df['current_date'] = pd.Series([now.date() for x in range(len(data_df.index))])
data_df['current_date'] = data_df['current_date'].astype('datetime64[ns]')
data_df['credit_line_age'] = (data_df['current_date'] - data_df['earliest_cr_line']) / np.timedelta64(1, 'D')
data_df['loan_age'] = (data_df['current_date'] - data_df['issue_d']) / np.timedelta64(1, 'D')
data_df.head()


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,...,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address,emp_duration,current_date,credit_line_age,loan_age
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,RENT,117000.0,Not Verified,...,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690",10,2023-01-28,11929.0,2949.0
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,MORTGAGE,65000.0,Not Verified,...,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113",4,2023-01-28,6785.0,2949.0
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,RENT,43057.0,Source Verified,...,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113",0,2023-01-28,5659.0,2949.0
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,RENT,54000.0,Not Verified,...,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813",6,2023-01-28,5993.0,3010.0
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,MORTGAGE,55000.0,Verified,...,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650",9,2023-01-28,8734.0,3589.0


In [69]:
data_df.drop(['current_date', 'earliest_cr_line', 'issue_d'], axis='columns', inplace=True)

In [71]:
# Drop the address column as it may not have much significant for our analysis
data_df.drop(['address'], axis='columns', inplace=True)

In [74]:
# Let's drop the 'title' column as well
data_df.drop(['title'], axis='columns', inplace=True)

In [75]:
# Segregate Categorical Features
cat_features = list(data_df.select_dtypes('object').columns)
for col in cat_features:
  print('Unique values in {0} are {1}'.format(col, data_df[col].unique()))

Unique values in term are [' 36 months' ' 60 months']
Unique values in grade are ['B' 'A' 'C' 'E' 'D' 'F' 'G']
Unique values in sub_grade are ['B4' 'B5' 'B3' 'A2' 'C5' 'C3' 'A1' 'B2' 'C1' 'A5' 'E4' 'A4' 'A3' 'D1'
 'C2' 'B1' 'D3' 'D5' 'D2' 'E1' 'E2' 'E5' 'F4' 'E3' 'D4' 'G1' 'F5' 'G2'
 'C4' 'F1' 'F3' 'G5' 'G4' 'F2' 'G3']
Unique values in emp_title are ['Marketing' 'Credit analyst ' 'Statistician' ...
 "Michael's Arts & Crafts" 'licensed bankere' 'Gracon Services, Inc']
Unique values in home_ownership are ['RENT' 'MORTGAGE' 'OWN' 'OTHER' 'ANY' 'NONE']
Unique values in verification_status are ['Not Verified' 'Source Verified' 'Verified']
Unique values in loan_status are ['Fully Paid' 'Charged Off']
Unique values in purpose are ['vacation' 'debt_consolidation' 'credit_card' 'home_improvement'
 'small_business' 'major_purchase' 'other' 'medical' 'wedding' 'car'
 'moving' 'house' 'educational' 'renewable_energy']
Unique values in initial_list_status are ['w' 'f']
Unique values in application_

In [76]:
# Sub grade is secondary level categorization of grade, so we can ignore the first part and convert it to int type
data_df['sub_grade'] = data_df['sub_grade'].str[1:]

In [78]:
data_df['sub_grade'] = data_df['sub_grade'].astype(int)

In [2]:
# Segregate Numerical Features
num_features = data_df.select_dtypes('number')


NameError: ignored

In [1]:
# Check for Outliers
num_features.columns

NameError: ignored